In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import Dataset, load_dataset
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler
from tqdm import tqdm
import evaluate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import random
import pandas as pd
from collections import Counter
import pickle
import joblib
from copy import deepcopy


sns.set_style('whitegrid')

import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

violet = '#702b9d'
RANDOM_SEED = 1002

checkpoint = 'FacebookAI/xlm-roberta-base'

In [2]:
def convert_train(example):
    with open(os.getcwd() + '/data_en/en.intents', 'r', encoding = 'UTF-8') as file:
        intent_labels_map = json.load(file)
    
    with open(os.getcwd() + '/data_en/en.slots', 'r', encoding = 'UTF-8') as file:
        slot_labels_map = json.load(file)
    
        
    label_to_pred_intent_idx = {v: k for k, v in intent_labels_map.items()}
    
    label_to_pred_slot_idx = {v: k for k, v in slot_labels_map.items()}
    
    converted_intent_labels = [int(label_to_pred_intent_idx.get(p, -100)) for p in example['target_intents']]  
    converted_slot_labels = [[int(label_to_pred_slot_idx.get(s, -100)) for s in slot_seq] for slot_seq in example['target_slots']]
    example['target_intents'], example['target_slots'] = converted_intent_labels, converted_slot_labels
    return example
       

In [3]:
en_train = Dataset.from_file(os.getcwd() + '/data_en/en.train/data-00000-of-00001.arrow')
zh_train = Dataset.from_file(os.getcwd() + '/data_zh/zh.train/data-00000-of-00001.arrow')
    
zh_val = Dataset.from_file(os.getcwd() + '/data_zh/zh.dev/data-00000-of-00001.arrow')
para_dataset = deepcopy(en_train)
para_dataset = para_dataset.add_column("target_utt", zh_train['utt'])
para_dataset = para_dataset.add_column("target_slots", zh_train['slots_str'])
para_dataset = para_dataset.add_column("target_intents", zh_train['intent_str'])
para_dataset = para_dataset.map(lambda x: convert_train(x), batched=True)

In [4]:
from utils import *  

tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
batch_size = 32

para_dataloader = DataLoader(para_dataset, batch_size= batch_size, shuffle=True, 
                            collate_fn=CollatorMASSIVEIntentClassSlotFill_para(tokenizer=tokenizer, max_length=512))

In [ ]:
!python main_soft_align.py --train --num_epochs 1

-100